In [42]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import re
!pip install pandas
import pandas as pd
import numpy as np


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\David Pineda\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [43]:
# Constantes
MAX_JORNADAS_1 = 38
URL_PRIMERA_2022 = "https://www.resultados-futbol.com/premier/grupo1/jornada%s"
TEMPORADA_2022 = '2023-2024'


In [44]:
# Clases
class Equipo:
    def __init__(self, id_club, nombre):
        self.id_club = id_club
        self.nombre = nombre

    def __str__(self):
        return f"{self.id_club} - {self.nombre}"


class Partido:
    def __init__(self, id_partido, temporada, division, jornada, local,
                 visitante, goles_local, goles_visitante, fecha):
        self.id_partido = id_partido
        self.temporada = temporada
        self.division = division
        self.jornada = jornada
        self.local = local
        self.visitante = visitante
        self.goles_local = goles_local
        self.goles_visitante = goles_visitante
        self.fecha = fecha
        self.timestamp = self.convertir_fecha_a_timestamp(fecha)

    def __str__(self):
        return f"{self.id_partido}::{self.temporada}::{self.division}::{self.jornada}::" \
               f"{self.local}::{self.visitante}::{self.goles_local}::{self.goles_visitante}::" \
               f"{self.fecha}::{self.timestamp}"

    @staticmethod
    def convertir_fecha_a_timestamp(fecha):
        dt = datetime.strptime(fecha, "%d/%m/%Y")
        return int(dt.timestamp())

def crear_diccionario_partidos(partidos):
    diccionario_partidos = {}
    for partido in partidos:
        local = Equipo(partido["local"]["id_club"], partido["local"]["nombre"])
        visitante = Equipo(partido["visitante"]["id_club"], partido["visitante"]["nombre"])
        id_partido = partido["id_partido"]
        temporada = partido["temporada"]
        division = partido["division"]
        jornada = partido["jornada"]
        goles_local = partido["goles_local"]
        goles_visitante = partido["goles_visitante"]
        fecha = partido["fecha"]
        nuevo_partido = Partido(id_partido, temporada, division, jornada, local, visitante, goles_local, goles_visitante, fecha)
        diccionario_partidos[id_partido] = nuevo_partido
    return diccionario_partidos

In [45]:
# En este fichero voy a obtener los resultados de futbol de la temporada 2014-15
# WEB: http://www.resultados-futbol.com
def get_fecha_partido(fecha_sucia):
    charts_remove = ['[', '\\n', '\\t']
    fecha = re.sub(r'[<](/)?td[^>]*[>]', '', fecha_sucia).translate(str.maketrans('', '', ''.join(charts_remove)))
    fecha = fecha.split('<br/>')[0].strip()
    fecha = fecha.split(' ')
    meses = {'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
             'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12', 'Oc': '10', 'Ee': '01', 'Ju': '06'}
    fecha = f"{fecha[0]}/{meses[fecha[1]]}/20{fecha[2]}"
    return fecha

# Obtengo el nombre del equipo de fútbol
def get_equipo(equipo_sucio):
    club = equipo_sucio.find_all('a')[1].attrs['href'].replace('/', '')
    return replace_equipos(club.strip())

# Obtengo el resultado
def get_resultado(resultado_sucio):
    resultado = resultado_sucio.find('a').text
    resultado = re.sub(r'[<](/)?a[^>]*[>]', '', resultado).replace('</a>', '').strip()
    if 'x' in resultado:
        return None
    else:
        return resultado.split('-')

# Contador de Partidos
contador = 0

# Obtengo los datos de un partido de fútbol
def get_partido(tr_partido):
    soup_tr = BeautifulSoup(tr_partido, "html.parser")

    fecha = get_fecha_partido(soup_tr.find('td', {'class': 'fecha'}).text)
    local = get_equipo(soup_tr.find('td', {'class': 'equipo1'}))
    visitante = get_equipo(soup_tr.find('td', {'class': 'equipo2'}))
    resultado = get_resultado(soup_tr.find('td', {'class': 'rstd'}))

    return {
        "local": local,
        "visitante": visitante,
        "gLocal": resultado[0],
        "gVisitante": resultado[1],
        "fecha": fecha
    }

# Obtengo un listado con los partidos de Fútbol
def find_partidos(tabla_partidos):
    partidos_jornada = []
    soup_tabla = BeautifulSoup(tabla_partidos, "html.parser")
    tr_partidos = soup_tabla.find_all('tr', {'class': 'vevent'})
    
    for tr in tr_partidos:
        partido = {
            'local': tr.find('td', {'class': 'equipo1'}).find('a').text.strip() if tr.find('td', {'class': 'equipo1'}) is not None else '',
            'visitante': tr.find('td', {'class': 'equipo2'}).find('a').text.strip() if tr.find('td', {'class': 'equipo2'}) is not None else '',
            'gLocal': tr.find('td', {'class': 'rstd'}).text.strip() if tr.find('td', {'class': 'rstd'}) is not None else '',
            'gVisitante': tr.find('td', {'class': 'rstd'}).text.strip() if tr.find('td', {'class': 'rstv'}) is not None else '',
            'fecha': tr.find('td', {'class': 'fecha'}).text.strip() if tr.find('td', {'class': 'fecha'}) is not None else ''
        }
        partidos_jornada.append(partido)

    return partidos_jornada


# Obtengo un diccionario con los partidos de futbol obtenidos de esta temporada
def get_partidos():
    print ("****** PROCESANDO TEMPORADA 2022-2023 ******")

    partidos_list = []

    # Obtengo los partidos de primera division
    for i in range(1, MAX_JORNADAS_1 + 1):
        # Construyo las URLs
        url = URL_PRIMERA_2022 % i
        print ("Procesando %s" % url)

        # Realizo las peticiones a las URLs
        req_primera = requests.get(url)

        # Paso la request a un objeto BeautifulSoup
        soup_primera = BeautifulSoup(req_primera.text, "html.parser")

        tabla_partidos = str(soup_primera.find('table', {'id': 'tabla1'}))
        partidos_jornada = find_partidos(tabla_partidos)

        # Verificar si la lista de partidos es None
        if partidos_jornada is not None:
            for part in partidos_jornada:
                partido = {
                    'Id': len(partidos_list) + 1,
                    'Temporada': TEMPORADA_2022,
                    'Jornada': i,
                    'Equipo Local': part.get('local', ''),
                    'Equipo Visitante': part.get('visitante', ''),
                    'Goles Local': part.get('gLocal', ''),
                    'Goles Visitante': part.get('gVisitante', ''),
                    'Fecha': part.get('fecha', '')
                }
                partidos_list.append(partido)

    return partidos_list


In [46]:
partidos = get_partidos()

# Agrupar partidos por jornada
partidos_por_jornada = {}
for partido in partidos:
    jornada = partido['Jornada']
    if jornada not in partidos_por_jornada:
        partidos_por_jornada[jornada] = []
    partidos_por_jornada[jornada].append(partido)



****** PROCESANDO TEMPORADA 2022-2023 ******
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada1
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada2
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada3
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada4
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada5
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada6
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada7
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada8
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada9
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada10
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada11
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada12
Procesando https://www.resultados-futbol.com/premier/grupo1/jornada13
Procesando https://www.resultados-futbol.com/premier/g

In [47]:
# Mostrar resultados por jornada
for jornada, partidos in partidos_por_jornada.items():
    print(f"Jornada {jornada}")
    for partido in partidos:
        print(f"{partido['Equipo Local']} {partido['Goles Local']}-{partido['Goles Visitante']} {partido['Equipo Visitante']}")
    print()

Jornada 1
 Burnley - Man. City
2023-08-11T21:00:00
Turf Moor

21:00- 
 Arsenal - Nottingham Forest
2023-08-12T13:30:00
Emirates Stadium

13:30- 
 AFC Bournemouth - West Ham
2023-08-12T16:00:00
Vitality Stadium

16:00- 
 Brighton & Hove Albion - Luton Town
2023-08-12T16:00:00
The American Express Community Stadium

16:00- 
 Everton - Fulham
2023-08-12T16:00:00
Goodison Park

16:00- 
 Sheffield United - Crystal Palace
2023-08-12T16:00:00
Bramall Lane

16:00- 
 Newcastle - Aston Villa
2023-08-12T18:30:00
St. James' Park

18:30- 
 Brentford - Tottenham Hotspur
2023-08-13T15:00:00
Griffin Park

15:00- 
 Chelsea - Liverpool
2023-08-13T17:30:00
Stamford Bridge

17:30- 
 Man. Utd - Wolves
2023-08-14T21:00:00
Old Trafford

21:00- 

Jornada 2
 Nottingham Forest - Sheffield United
2023-08-18T20:45:00
The City Ground

20:45- 
 Fulham - Brentford
2023-08-19T16:00:00
Craven Cottage

16:00- 
 Liverpool - AFC Bournemouth
2023-08-19T16:00:00
Anfield

16:00- 
 Luton Town - Burnley
2023-08-19T16:00:00
Ke

In [48]:
# Mostrar resultados por jornada
for jornada, partidos in partidos_por_jornada.items():
    display(f"Jornada {jornada}")
    for partido in partidos:
        display(f"{partido['Equipo Local']} {partido['Goles Local']}-{partido['Goles Visitante']} {partido['Equipo Visitante']}")
    display()

'Jornada 1'

' Burnley - Man. City\n2023-08-11T21:00:00\nTurf Moor\n\n21:00- '

' Arsenal - Nottingham Forest\n2023-08-12T13:30:00\nEmirates Stadium\n\n13:30- '

' AFC Bournemouth - West Ham\n2023-08-12T16:00:00\nVitality Stadium\n\n16:00- '

' Brighton & Hove Albion - Luton Town\n2023-08-12T16:00:00\nThe American Express Community Stadium\n\n16:00- '

...

' Crystal Palace - Aston Villa\n2024-05-19T17:00:00\nSelhurst Park\n\n17:00- '

' Liverpool - Wolves\n2024-05-19T17:00:00\nAnfield\n\n17:00- '

' Luton Town - Fulham\n2024-05-19T17:00:00\nKenilworth Road Stadium\n\n17:00- '

' Man. City - West Ham\n2024-05-19T17:00:00\nEtihad Stadium\n\n17:00- '

' Sheffield United - Tottenham Hotspur\n2024-05-19T17:00:00\nBramall Lane\n\n17:00- '

In [49]:
# Convertir los datos a una lista de diccionarios
data = []
for jornada, partidos in partidos_por_jornada.items():
    for partido in partidos:
        data.append({
            'jornada': jornada,
            'local': partido['Equipo Local'],
            'visita': partido['Equipo Visitante'],
            'goles_local': partido['Goles Local'],
            'goles_visita': partido['Goles Visitante'],
            'fecha': partido['Fecha'],
            })

# Crear un DataFrame a partir de la lista de diccionarios
partidos_jornada = pd.DataFrame(data)

# Mostrar el DataFrame
display(partidos_jornada)

,jornada,local,visita,goles_local,goles_visita,fecha
0,1,,,Burnley - Man. City\n2023-08-11T21:00:00\nTurf...,,11 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
1,1,,,Arsenal - Nottingham Forest\n2023-08-12T13:30:...,,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
2,1,,,AFC Bournemouth - West Ham\n2023-08-12T16:00:0...,,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
3,1,,,Brighton & Hove Albion - Luton Town\n2023-08-1...,,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
4,1,,,Everton - Fulham\n2023-08-12T16:00:00\nGoodiso...,,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
...,...,...,...,...,...,...
375,38,,,Crystal Palace - Aston Villa\n2024-05-19T17:00...,,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
376,38,,,Liverpool - Wolves\n2024-05-19T17:00:00\nAnfie...,,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
377,38,,,Luton Town - Fulham\n2024-05-19T17:00:00\nKeni...,,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
378,38,,,Man. City - West Ham\n2024-05-19T17:00:00\nEti...,,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar


In [50]:
partidos_jornada.describe()

,jornada
count,380.000000
mean,19.500000
std,10.980313
min,1.000000
25%,10.000000
50%,19.500000
75%,29.000000
max,38.000000


In [51]:
partidos_jornada.dtypes

jornada          int64
local           object
visita          object
goles_local     object
goles_visita    object
fecha           object
dtype: object

In [52]:
partidos_jornada1 = partidos_jornada.drop(['local', 'visita', 'goles_visita'], axis=1)
display(partidos_jornada1)

,jornada,goles_local,fecha
0,1,Burnley - Man. City\n2023-08-11T21:00:00\nTurf...,11 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
1,1,Arsenal - Nottingham Forest\n2023-08-12T13:30:...,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
2,1,AFC Bournemouth - West Ham\n2023-08-12T16:00:0...,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
3,1,Brighton & Hove Albion - Luton Town\n2023-08-1...,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
4,1,Everton - Fulham\n2023-08-12T16:00:00\nGoodiso...,12 Ago 23\t\t\t\t\t\t\t\t\nSin comenzar
...,...,...,...
375,38,Crystal Palace - Aston Villa\n2024-05-19T17:00...,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
376,38,Liverpool - Wolves\n2024-05-19T17:00:00\nAnfie...,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
377,38,Luton Town - Fulham\n2024-05-19T17:00:00\nKeni...,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar
378,38,Man. City - West Ham\n2024-05-19T17:00:00\nEti...,19 May 24\t\t\t\t\t\t\t\t\nSin comenzar


In [53]:

partidos_jornada1[['equipos', 'fecha', 'lugar', 'marcador']] = partidos_jornada1['goles_local'].str.split('\n', n=3, expand=True)


In [33]:
print(len(partidos_jornada1['goles_local']))
print(len(partidos_jornada1['goles_local'].str.split('\n', expand=True)))

380
380


In [34]:
display(partidos_jornada1)

,jornada,goles_local,fecha,equipos,lugar,marcador
0,1,AlmerÃ­a - Rayo Vallecano\n2023-08-11T19:30:00...,2023-08-11T19:30:00,AlmerÃ­a - Rayo Vallecano,Juegos del MediterrÃ¡neo,\n19:30
1,1,Sevilla - Valencia\n2023-08-11T21:30:00\nRamÃ³...,2023-08-11T21:30:00,Sevilla - Valencia,RamÃ³n SÃ¡nchez PizjuÃ¡n,\n21:30
2,1,Real Sociedad - Girona\n2023-08-12T17:00:00\nA...,2023-08-12T17:00:00,Real Sociedad - Girona,Anoeta,\n17:00
3,1,Las Palmas - Mallorca\n2023-08-12T19:30:00\nGr...,2023-08-12T19:30:00,Las Palmas - Mallorca,Gran Canaria,\n19:30
4,1,Athletic - Real Madrid\n2023-08-12T21:30:00\nS...,2023-08-12T21:30:00,Athletic - Real Madrid,San MamÃ©s,\n21:30
...,...,...,...,...,...,...
375,38,Rayo Vallecano - Athletic\n2024-05-26T00:00:00...,2024-05-26T00:00:00,Rayo Vallecano - Athletic,Vallecas,\nx-x
376,38,Real Sociedad - AtlÃ©tico\n2024-05-26T00:00:00...,2024-05-26T00:00:00,Real Sociedad - AtlÃ©tico,Anoeta,\nx-x
377,38,Sevilla - Barcelona\n2024-05-26T00:00:00\nRamÃ...,2024-05-26T00:00:00,Sevilla - Barcelona,RamÃ³n SÃ¡nchez PizjuÃ¡n,\nx-x
378,38,Real Madrid - Real Betis\n2024-05-26T00:00:00\...,2024-05-26T00:00:00,Real Madrid - Real Betis,Santiago BernabÃ©u,\nx-x


In [35]:
partidos_jornada2=partidos_jornada1.drop(['goles_local'], axis=1)

display(partidos_jornada2)

,jornada,fecha,equipos,lugar,marcador
0,1,2023-08-11T19:30:00,AlmerÃ­a - Rayo Vallecano,Juegos del MediterrÃ¡neo,\n19:30
1,1,2023-08-11T21:30:00,Sevilla - Valencia,RamÃ³n SÃ¡nchez PizjuÃ¡n,\n21:30
2,1,2023-08-12T17:00:00,Real Sociedad - Girona,Anoeta,\n17:00
3,1,2023-08-12T19:30:00,Las Palmas - Mallorca,Gran Canaria,\n19:30
4,1,2023-08-12T21:30:00,Athletic - Real Madrid,San MamÃ©s,\n21:30
...,...,...,...,...,...
375,38,2024-05-26T00:00:00,Rayo Vallecano - Athletic,Vallecas,\nx-x
376,38,2024-05-26T00:00:00,Real Sociedad - AtlÃ©tico,Anoeta,\nx-x
377,38,2024-05-26T00:00:00,Sevilla - Barcelona,RamÃ³n SÃ¡nchez PizjuÃ¡n,\nx-x
378,38,2024-05-26T00:00:00,Real Madrid - Real Betis,Santiago BernabÃ©u,\nx-x


In [36]:
partidos_jornada2[['local', 'visita']] = partidos_jornada2['equipos'].str.split('-', expand=True)

partidos_jornada2['marcador'] = partidos_jornada2['marcador'].str.replace('\n', '')

partidos_jornada2[['c', 'v']] = partidos_jornada2['marcador'].str.split('-', expand=True)

display(partidos_jornada2)

,jornada,fecha,equipos,lugar,marcador,local,visita,c,v
0,1,2023-08-11T19:30:00,AlmerÃ­a - Rayo Vallecano,Juegos del MediterrÃ¡neo,19:30,AlmerÃ­a,Rayo Vallecano,19:30,None
1,1,2023-08-11T21:30:00,Sevilla - Valencia,RamÃ³n SÃ¡nchez PizjuÃ¡n,21:30,Sevilla,Valencia,21:30,None
2,1,2023-08-12T17:00:00,Real Sociedad - Girona,Anoeta,17:00,Real Sociedad,Girona,17:00,None
3,1,2023-08-12T19:30:00,Las Palmas - Mallorca,Gran Canaria,19:30,Las Palmas,Mallorca,19:30,None
4,1,2023-08-12T21:30:00,Athletic - Real Madrid,San MamÃ©s,21:30,Athletic,Real Madrid,21:30,None
...,...,...,...,...,...,...,...,...,...
375,38,2024-05-26T00:00:00,Rayo Vallecano - Athletic,Vallecas,x-x,Rayo Vallecano,Athletic,x,x
376,38,2024-05-26T00:00:00,Real Sociedad - AtlÃ©tico,Anoeta,x-x,Real Sociedad,AtlÃ©tico,x,x
377,38,2024-05-26T00:00:00,Sevilla - Barcelona,RamÃ³n SÃ¡nchez PizjuÃ¡n,x-x,Sevilla,Barcelona,x,x
378,38,2024-05-26T00:00:00,Real Madrid - Real Betis,Santiago BernabÃ©u,x-x,Real Madrid,Real Betis,x,x


In [37]:
partidos_jornadaok= partidos_jornada2.drop(['equipos','marcador'], axis=1)

display(partidos_jornadaok[['jornada','fecha','lugar','local','c','v','visita']])

,jornada,fecha,lugar,local,c,v,visita
0,1,2023-08-11T19:30:00,Juegos del MediterrÃ¡neo,AlmerÃ­a,19:30,None,Rayo Vallecano
1,1,2023-08-11T21:30:00,RamÃ³n SÃ¡nchez PizjuÃ¡n,Sevilla,21:30,None,Valencia
2,1,2023-08-12T17:00:00,Anoeta,Real Sociedad,17:00,None,Girona
3,1,2023-08-12T19:30:00,Gran Canaria,Las Palmas,19:30,None,Mallorca
4,1,2023-08-12T21:30:00,San MamÃ©s,Athletic,21:30,None,Real Madrid
...,...,...,...,...,...,...,...
375,38,2024-05-26T00:00:00,Vallecas,Rayo Vallecano,x,x,Athletic
376,38,2024-05-26T00:00:00,Anoeta,Real Sociedad,x,x,AtlÃ©tico
377,38,2024-05-26T00:00:00,RamÃ³n SÃ¡nchez PizjuÃ¡n,Sevilla,x,x,Barcelona
378,38,2024-05-26T00:00:00,Santiago BernabÃ©u,Real Madrid,x,x,Real Betis


In [38]:
def get_resultado(row):
    c = row['c']
    v = row['v']
    
    if pd.isna(c) or pd.isna(v):
        return None
    
    if isinstance(c, str) or isinstance(v, str):
        return None
    
    if c > v:
        return 1
    elif c < v:
        return 2
    else:
        return 0.5

partidos_jornadaok['resultado'] = partidos_jornadaok.apply(get_resultado, axis=1)

display(partidos_jornadaok)

,jornada,fecha,lugar,local,visita,c,v,resultado
0,1,2023-08-11T19:30:00,Juegos del MediterrÃ¡neo,AlmerÃ­a,Rayo Vallecano,19:30,None,None
1,1,2023-08-11T21:30:00,RamÃ³n SÃ¡nchez PizjuÃ¡n,Sevilla,Valencia,21:30,None,None
2,1,2023-08-12T17:00:00,Anoeta,Real Sociedad,Girona,17:00,None,None
3,1,2023-08-12T19:30:00,Gran Canaria,Las Palmas,Mallorca,19:30,None,None
4,1,2023-08-12T21:30:00,San MamÃ©s,Athletic,Real Madrid,21:30,None,None
...,...,...,...,...,...,...,...,...
375,38,2024-05-26T00:00:00,Vallecas,Rayo Vallecano,Athletic,x,x,None
376,38,2024-05-26T00:00:00,Anoeta,Real Sociedad,AtlÃ©tico,x,x,None
377,38,2024-05-26T00:00:00,RamÃ³n SÃ¡nchez PizjuÃ¡n,Sevilla,Barcelona,x,x,None
378,38,2024-05-26T00:00:00,Santiago BernabÃ©u,Real Madrid,Real Betis,x,x,None


In [39]:
partidos_jornadaok['c'] = pd.to_numeric(partidos_jornadaok['c'], errors='coerce')
partidos_jornadaok['v'] = pd.to_numeric(partidos_jornadaok['v'], errors='coerce')

In [40]:
def get_resultado(row):
    c = row['c']
    v = row['v']

    if pd.isna(c) and pd.isna(v):
        return None

    if isinstance(c, str) or isinstance(v, str):
        return None

    if c > v:
        return 1
    elif c < v:
        return 2
    else:
        return 0.5

partidos_jornadaok['resultados'] = None
partidos_jornadaok['resultados'] = partidos_jornadaok.apply(get_resultado, axis=1)

display(partidos_jornadaok[['jornada','fecha','local','c','v','visita','resultados']])

,jornada,fecha,local,c,v,visita,resultados
0,1,2023-08-11T19:30:00,AlmerÃ­a,NaN,NaN,Rayo Vallecano,None
1,1,2023-08-11T21:30:00,Sevilla,NaN,NaN,Valencia,None
2,1,2023-08-12T17:00:00,Real Sociedad,NaN,NaN,Girona,None
3,1,2023-08-12T19:30:00,Las Palmas,NaN,NaN,Mallorca,None
4,1,2023-08-12T21:30:00,Athletic,NaN,NaN,Real Madrid,None
...,...,...,...,...,...,...,...
375,38,2024-05-26T00:00:00,Rayo Vallecano,NaN,NaN,Athletic,None
376,38,2024-05-26T00:00:00,Real Sociedad,NaN,NaN,AtlÃ©tico,None
377,38,2024-05-26T00:00:00,Sevilla,NaN,NaN,Barcelona,None
378,38,2024-05-26T00:00:00,Real Madrid,NaN,NaN,Real Betis,None


In [41]:
# guardar los resultados en un archivo excel
partidos_jornadaok.to_excel('ligaespanola2024.xlsx', index=False)